In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from datetime import date
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras import Sequential
import tensorflow as tf
import yfinance as yf
from sklearn.model_selection import GridSearchCV

In [5]:

def loadTrainingData(ticker, START, TODAY):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

In [6]:
START = "2010-01-01"
TODAY = date.today().strftime("%Y-%m-%d")
data = loadTrainingData('AAPL',START,TODAY)
df = data
df = df.drop(['Date', 'Adj Close'], axis = 1)
df.head()
df=data
df.head()
x_train = []
y_train = []
df = df.iloc[:, 4:5].values
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)
for i in range(5, df.shape[0]):
    x_train.append(df[i - 5: i])
    y_train.append(df[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
print(x_train.shape)
print(y_train.shape)

[*********************100%%**********************]  1 of 1 completed

(3637, 5, 1)
(3637,)


In [7]:
def create_model(optimizer, loss):
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', return_sequences=True
                   , input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units=60, activation='relu', return_sequences=True))
    model.add(Dropout(0.3))

    model.add(LSTM(units=80, activation='relu', return_sequences=True))
    model.add(Dropout(0.4))

    model.add(LSTM(units=120, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(units=1))
    model.compile(optimizer=optimizer, loss=loss)
    return model
def hyperParamTuning(x_train,y_train):
    tf.compat.v1.disable_eager_execution()
    model = KerasClassifier(build_fn=create_model, epochs= 10, verbose=1)
    # Definizione della grid dei parametri
    param_grid = {
        'optimizer': ['adam','adamax'],
        'loss': ['mean_squared_error', 'mean_squared_logarithmic_error'],
    }
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3, scoring = 'r2', verbose=2)
    grid_result = grid.fit(x_train, y_train)
    print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
hyperParamTuning(x_train,y_train)

C:\Users\gabri\AppData\Local\Temp\ipykernel_26928\996535980.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs= 10, verbose=1)


Fitting 3 folds for each of 4 candidates, totalling 12 fits
Train on 2424 samples
Epoch 1/10
2424/2424 [==============================] - 3s 1ms/sample - loss: 1131925.2349
Epoch 2/10
2424/2424 [==============================] - 1s 442us/sample - loss: 115078.4323
Epoch 3/10
2424/2424 [==============================] - 1s 454us/sample - loss: 106023.1075
Epoch 4/10
2424/2424 [==============================] - 1s 438us/sample - loss: 87799.1290
Epoch 5/10
2424/2424 [==============================] - 1s 436us/sample - loss: 73206.3900
Epoch 6/10
2424/2424 [==============================] - 1s 452us/sample - loss: 66286.6515
Epoch 7/10
2424/2424 [==============================] - 1s 489us/sample - loss: 53748.8485
Epoch 8/10
2424/2424 [==============================] - 1s 473us/sample - loss: 51681.1323
Epoch 9/10
2424/2424 [==============================] - 1s 484us/sample - loss: 49959.9806
Epoch 10/10
2424/2424 [==============================] - 1s 479us/sample - loss: 50024.8160


C:\Users\gabri\.conda\envs\tf\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[CV] END ............loss=mean_squared_error, optimizer=adam; total time=  16.0s
Train on 2425 samples
Epoch 1/10
2425/2425 [==============================] - 3s 1ms/sample - loss: 1196356.6572
Epoch 2/10
2425/2425 [==============================] - 1s 471us/sample - loss: 128008.3871
Epoch 3/10
2425/2425 [==============================] - 1s 485us/sample - loss: 124142.3165
Epoch 4/10
2425/2425 [==============================] - 1s 477us/sample - loss: 107980.2610
Epoch 5/10
2425/2425 [==============================] - 1s 480us/sample - loss: 104261.0804
Epoch 6/10
2425/2425 [==============================] - 1s 476us/sample - loss: 86052.2313
Epoch 7/10
2425/2425 [==============================] - 1s 500us/sample - loss: 75534.0779
Epoch 8/10
2425/2425 [==============================] - 1s 516us/sample - loss: 62028.6371
Epoch 9/10
2425/2425 [==============================] - 1s 518us/sample - loss: 54869.8614
Epoch 10/10
2425/2425 [==============================] - 1s 516us/sample -

C:\Users\gabri\.conda\envs\tf\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


...

[CV] END loss=mean_squared_logarithmic_error, optimizer=adamax; total time=  24.8s
Train on 2425 samples
Epoch 1/10
2425/2425 [==============================] - 8s 3ms/sample - loss: 31.6206
Epoch 2/10
2425/2425 [==============================] - 1s 507us/sample - loss: 0.6467
Epoch 3/10
2425/2425 [==============================] - 1s 531us/sample - loss: 0.5198
Epoch 4/10
2425/2425 [==============================] - 1s 496us/sample - loss: 0.4993
Epoch 5/10
2425/2425 [==============================] - 1s 490us/sample - loss: 0.4815
Epoch 6/10
2425/2425 [==============================] - 1s 489us/sample - loss: 0.4722
Epoch 7/10
2425/2425 [==============================] - 1s 518us/sample - loss: 0.4674
Epoch 8/10
2425/2425 [==============================] - 1s 498us/sample - loss: 0.4573
Epoch 9/10
2425/2425 [==============================] - 1s 502us/sample - loss: 0.4458
Epoch 10/10
2425/2425 [==============================] - 1s 490us/sample - loss: 0.4373


C:\Users\gabri\.conda\envs\tf\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[CV] END loss=mean_squared_logarithmic_error, optimizer=adamax; total time=  24.6s
Train on 2425 samples
Epoch 1/10
2425/2425 [==============================] - 9s 4ms/sample - loss: 31.7354
Epoch 2/10
2425/2425 [==============================] - 1s 505us/sample - loss: 0.9761
Epoch 3/10
2425/2425 [==============================] - 1s 543us/sample - loss: 0.8177
Epoch 4/10
2425/2425 [==============================] - 1s 557us/sample - loss: 0.7461
Epoch 5/10
2425/2425 [==============================] - 1s 563us/sample - loss: 0.6633
Epoch 6/10
2425/2425 [==============================] - 1s 570us/sample - loss: 0.5132
Epoch 7/10
2425/2425 [==============================] - 1s 582us/sample - loss: 0.3916
Epoch 8/10
2425/2425 [==============================] - 1s 594us/sample - loss: 0.3027
Epoch 9/10
2425/2425 [==============================] - 1s 587us/sample - loss: 0.2660
Epoch 10/10
2425/2425 [==============================] - 1s 597us/sample - loss: 0.2336


C:\Users\gabri\.conda\envs\tf\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[CV] END loss=mean_squared_logarithmic_error, optimizer=adamax; total time=  27.0s
Train on 3637 samples
Epoch 1/10
3637/3637 [==============================] - 11s 3ms/sample - loss: 1775033.9172
Epoch 2/10
3637/3637 [==============================] - 2s 606us/sample - loss: 355533.0467
Epoch 3/10
3637/3637 [==============================] - 2s 610us/sample - loss: 340917.8134
Epoch 4/10
3637/3637 [==============================] - 2s 646us/sample - loss: 337388.7986
Epoch 5/10
3637/3637 [==============================] - 2s 608us/sample - loss: 330239.9704
Epoch 6/10
3637/3637 [==============================] - 2s 611us/sample - loss: 325681.6530
Epoch 7/10
3637/3637 [==============================] - 2s 646us/sample - loss: 217555.4089
Epoch 8/10
3637/3637 [==============================] - 2s 614us/sample - loss: 117777.2823
Epoch 9/10
3637/3637 [==============================] - 2s 595us/sample - loss: 104581.2873
Epoch 10/10
3637/3637 [==============================] - 2s 623us/s